In [1]:
pip install sastrawi

/bin/bash: /home/anggapark/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import re
import random
# from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [2]:
course = pd.read_csv('course.csv')
problem = pd.read_csv('problemstate.csv')

In [3]:
course

,courseID,judul,deskripsi,dampak,modul
0,1,Fundamental Course (1),Fundamental Attitude toward sustainability,mengubah paradigma mengenai sustainability; Me...,Perubahan menuju sustainable mindset; 5 Prinsi...
1,2,Fundamental Course (2),Innovation method as problem solving,Memahami konsep Growth Mindset; Memahami konse...,Growth mindset; Design thinking; Lean Startup;...
2,3,Fundamental Course (3),The way to build sustainable business,Mengenal Sustainable Startup; Memahami SDGs po...,Mengenal Sustainable Startup; SDGs 12 Responsi...
3,4,Indonesia Sustainability Coral Reef University...,Terumbu karang menutupi kurang dari 1% wilayah...,pelindungan habibat; mengurangi polusi; mengur...,Sustainability Leadership; Coral Reef Ecology;...
4,5,Ecotourism,Seiring dengan pelonggaran perjalanan dan stab...,Pelestarian ekosistem terhadap biodiversitas; ...,Ecoturism Introduction; Sustainable Tourism Pr...
5,6,Moringa Academy,Moringa Oliefera (kelor) merupakan salah satu ...,"Mengembangkan, memelihara dan melindungi hutan...",Pengenalan Kelor; Pemahaman Kelor Melalui sosi...
6,7,Indonesia Sustainable Social Forestry Educatio...,"Program ini terkait dengan potensi ""perhutanan...",Pemasaran produk-produk hutan yang Berkelanjut...,Forest Management and Conservation Program (FM...
7,8,Waste Management,"Bukan hanya kota yang darurat sampah, Desa Des...",pelindungan habitat; mengurangi polusi sampah ...,Eco Literasi (Perubahan Mindset; 5 Prinsip Eko...
8,9,Integrated Farming,Sistem pertanian terpadu merupakan sistem inte...,Menjaga keseimbangan ekosistem di dalamnya\nse...,Analisis Masalah Sistem pertanian dan pengenal...
9,10,Solar Academy,Pemerintah Indonesia saat ini memiliki target ...,Memberikan pengetahuan dan informasi\nterkini ...,Dasar - dasar solar PV\r\nPemahaman dasar rang...


In [4]:
# periksa sampel teks
def sample_text(df, idx):
    for col in df.columns[1:]:
        print(f'{col}:')
        print(df[col].iloc[idx])
        print('\n')
        
sample_text(course, 3)

judul:
Indonesia Sustainability Coral Reef University Network (ISCORE)


deskripsi:
Terumbu karang menutupi kurang dari 1% wilayah lautan, tetapi mereka secara langsung mendukung jutaan orang dengan
menyediakan makanan, pekerjaan, dan sumber daya lainnya. Bahkan lebih banyak orang yang mendapat manfaat dari
terumbu karang secara tidak langsung; ekosistem ini membantu memberi makan 1 miliar orang di Asia saja.
Populasi manusia hampir 7 miliar orang, dan kemungkinan akan tumbuh menjadi 9 miliar pada tahun 2040. Kita
membutuhkan terumbu karang (dan ekosistem lainnya) untuk memasok lebih banyak sumber daya untuk mendukung
jumlah kita yang terus bertambah, tetapi mereka semakin terancam dengan kehancuran .


dampak:
pelindungan habibat; mengurangi polusi; mengurangi dampak erosi; promosi pariwisata; meningkatkan komunitas lokal; pemeliharaan agen perubahan berkelanjutan dan perlindungan terumbu karang


modul:
Sustainability Leadership; Coral Reef Ecology; Human threats & Challenges; Integr

In [5]:
course.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   courseID   11 non-null     int64 
 1   judul      11 non-null     object
 2   deskripsi  11 non-null     object
 3   dampak     11 non-null     object
 4   modul      11 non-null     object
dtypes: int64(1), object(4)
memory usage: 568.0+ bytes


In [6]:
problem

,desa,problem,lokasi
0,desa pinogu,Budidaya dan pengolahan kopi,gorontalo
1,desa pinogu,Pengolahan produk dari sapi,gorontalo
2,desa pinogu,Akses jalan yang buruk,gorontalo
3,desa pinogu,Listrik yang kurang menyeluruh,gorontalo
4,desa pinogu,Jaringan internet yang kurang lancar,gorontalo
...,...,...,...
78,desa aimoli,Wisata kuliner yang terbatas,alor
79,desa aimoli,Akses informasi wisata yang masih terbatas,alor
80,desa aimoli,Faslitas penunjang yang masih terbatas,alor
81,desa aimoli,"Belum ada inovasi pengelolaan SDA, seperti mak...",alor


In [7]:
problem.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   desa     83 non-null     object
 1   problem  83 non-null     object
 2   lokasi   83 non-null     object
dtypes: object(3)
memory usage: 2.1+ KB


In [9]:
# # agregat tiap masalah pada 1 desa menjadi list dalam 1 baris
# problem_agg = (problem.groupby(['desa','lokasi'])
#           .agg({'problem': lambda x: ", ".join(x)})
#           .rename({'Case' : 'Avg_Cases','Delivery' : 'Avg_Delivery'},axis=1)
#           .reset_index())

In [27]:
# problem_agg

## Stemming & Remove Stopwords

Stemming = mengubah kata imbuhan menjadi kata dasar </br>
Stopwords = Stop list ini berisi daftar kata umum yang mempunyai fungsi tapi tidak mempunyai art

In [9]:
course_clean = course.copy()
prob_clean = problem.copy()

In [10]:
# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# variable to clear symbols
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z #+_]')

# stopword
f = open("tala_stopword.txt", "r")
stopword_list = []
for line in f:
    stripped_line = line.strip()
    line_list = stripped_line.split()
    stopword_list.append(line_list[0])
f.close()

len(stopword_list)

705

In [12]:
def clean_text(text):
    text = stemmer.stem(text)
    text = text.lower() # lowercase text
    text = clean_spcl.sub(' ', text)
    text = clean_symbol.sub('', text)
    text = ' '.join(word for word in text.split() if word not in stopword_list) # hapus stopword dari kolom deskripsi
    return text

In [13]:
course_clean['deskripsi'] = course_clean['deskripsi'].apply(clean_text)
course_clean['dampak'] = course_clean['dampak'].apply(clean_text)
course_clean['modul'] = course_clean['modul'].apply(clean_text)
prob_clean['problem'] = problem['problem'].apply(clean_text)

In [14]:
sample_text(course_clean, 3)

judul:
Indonesia Sustainability Coral Reef University Network (ISCORE)


deskripsi:
terumbu karang tutup 1 wilayah laut langsung dukung juta orang sedia makan kerja sumber daya bahkan banyak orang manfaat terumbu karang langsung ekosistem bantu makan 1 miliar orang asia populasi manusia 7 miliar orang akan tumbuh 9 miliar 2040 butuh terumbu karang ekosistem pasok banyak sumber daya dukung ancam hancur


dampak:
lindung habibat polusi dampak erosi promosi pariwisata tingkat komunitas lokal pelihara agen ubah lindung terumbu karang


modul:
sustainability leadership coral reef ecology human threats challenges integrating sustainability in coral reef improving reef management by sustainable financing for communities benefit




In [15]:
prob_clean['problem'].iloc[0]

'budidaya olah kopi'

## TF-IDF dan Cosine Similarity

In [16]:
# mengubah kalimat menjadi angka berupa matriks
def conv_to_tf_idf(df, set_to_idx, col_transform):
    df.set_index(set_to_idx, inplace=True)
    tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0)
    tfidf_matrix = tf.fit_transform(col_transform)
    cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return cos_sim

In [17]:
cos_sim_course = conv_to_tf_idf(course_clean, 'judul', course_clean['deskripsi'])
cos_sim_prob = conv_to_tf_idf(prob_clean, 'desa', prob_clean['problem'])

In [18]:
cos_sim_course

array([[1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.0410249 ,
        0.0410249 ],
       [0.        , 0.        , 0.        , 1.        , 0.03207991,
        0.00711276, 0.00759292, 0.00450875, 0.00767963, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.03207991, 1.        ,
        0.01174512, 0.00258723, 0.02030233, 0.00464064, 0.02612287,
        0.02612287],
       [0.        , 0.        , 0.        , 0.00711276, 0.01174512,
        1.        , 0.0476343 , 0.02042758, 0.01345511, 0.02190086,
        0.02190086],
       [0.        , 0.        , 0.        , 0.00759292, 0.

In [19]:
cos_sim_prob

array([[1.        , 0.09858882, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.09858882, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.02368081],
       [0.        , 0.        , 0.        , ..., 0.        , 0.02368081,
        1.        ]])

In [20]:
indices = pd.Series(course_clean.index)
indices

0                                Fundamental Course (1)
1                                Fundamental Course (2)
2                                Fundamental Course (3)
3     Indonesia Sustainability Coral Reef University...
4                                            Ecotourism
5                                       Moringa Academy
6     Indonesia Sustainable Social Forestry Educatio...
7                                      Waste Management
8                                    Integrated Farming
9                                         Solar Academy
10                                      Program Startup
Name: judul, dtype: object

In [21]:
def recommendations(name, cos_sim = cos_sim_prob):
    
    recommendation_desa = []
    
    # Mengambil nama hotel berdasarkan variabel indicies
    idx = indices[indices == name].index[0]

    # Membuat series berdasarkan skor kesamaan
    score_series = pd.Series(cos_sim_prob[idx]).sort_values(ascending = False)

    # mengambil index dan dibuat 10 baris rekomendasi terbaik
    top_10_indexes = list(score_series.iloc[1:len(score_series)].index)
    
    for i in top_10_indexes:
        recommendation_desa.append(list(prob_clean.index)[i])
        
    return recommendation_desa

In [35]:
recommendations('Moringa Academy')

['desa dataran hijau',
 'desa dataran hijau',
 'desa dataran hijau',
 'desa pinogu',
 'desa dataran hijau',
 'desa dataran hijau',
 'desa saleman',
 'desa saleman',
 'desa lewalu',
 'desa saleman',
 'desa pinogu',
 'desa saleman',
 'desa saleman',
 'desa saleman',
 'desa saleman',
 'desa saleman',
 'desa saleman',
 'desa saleman',
 'desa saleman',
 'desa saleman',
 'desa saleman',
 'desa saleman',
 'desa saleman',
 'desa lewalu',
 'desa lewalu',
 'desa saleman',
 'desa aimoli',
 'desa aimoli',
 'desa aimoli',
 'desa aimoli',
 'desa aimoli',
 'desa lewalu',
 'desa lewalu',
 'desa lewalu',
 'desa lewalu',
 'desa lewalu',
 'desa lewalu',
 'desa lewalu',
 'desa lewalu',
 'desa lewalu',
 'desa lewalu',
 'desa lewalu',
 'desa lewalu',
 'desa saleman',
 'desa dataran hijau',
 'desa saleman',
 'desa pinogu',
 'desa pinogu',
 'desa pinogu',
 'desa pinogu',
 'desa pinogu',
 'desa pinogu',
 'desa pinogu',
 'desa pinogu',
 'desa pinogu',
 'desa pinogu',
 'desa pinogu',
 'desa pinogu',
 'desa pinog

# Aggregate tiap problem di Desa

In [25]:
# agregat tiap masalah pada 1 desa menjadi list dalam 1 baris
problem_agg = (problem.groupby(['desa','lokasi'])
          .agg({'problem': lambda x: ", ".join(x)})
          .rename({'Case' : 'Avg_Cases','Delivery' : 'Avg_Delivery'},axis=1)
          .reset_index())

In [26]:
problem_agg

,desa,lokasi,problem
0,desa aimoli,alor,"Fasilitas penunjang wisata yang masih minim, W..."
1,desa dataran hijau,gorontalo,"Akses yang sulit, Infrastruktur, kendala listr..."
2,desa lewalu,alor,"Sulit mencari souvenir atau oleh-oleh, Sampah,..."
3,desa pinogu,gorontalo,"Budidaya dan pengolahan kopi, Pengolahan produ..."
4,desa saleman,maluku,"Budaya masyarakat buang sampah ke laut, Etika ..."


In [31]:
course_clean = course.copy()
prob_clean = problem.copy()

In [32]:
course_clean['deskripsi'] = course_clean['deskripsi'].apply(clean_text)
course_clean['dampak'] = course_clean['dampak'].apply(clean_text)
course_clean['modul'] = course_clean['modul'].apply(clean_text)
prob_clean['problem'] = problem_agg['problem'].apply(clean_text)

In [33]:
cos_sim_course = conv_to_tf_idf(course_clean, 'judul', course_clean['deskripsi'])
cos_sim_prob = conv_to_tf_idf(prob_clean, 'desa', prob_clean['problem'])

ValueError: np.nan is an invalid document, expected byte or unicode string.